# input data 전처리 함수

In [110]:
def tagging_pos_input_data(komoran, input_str):

    input_token_komoran = komoran.get_plain_text(input_str).split(' ')
    # print('코모란:', input_token_komoran)

    df_komoran = pd.DataFrame([x.split('/') for x in input_token_komoran], columns=['형태소', '주석'])

    # df_komoran
    
    # 정규식 패턴
    pattern_english = r'^[a-zA-Z]+$'
    pattern_number = r'\d+'
    pattern_symbol = r'[!@#$%^&*()_+-={}\[\]:;"\'<>,.?/~`|]'
    pattern_empty = r'\s+'

    # 제거할 형태소 리스트
    list_english = []
    list_number = []
    list_symbol = []
    list_empty = []

    ## 영어 제거
    for index, row in df_komoran.iterrows():
        if re.match(pattern_english, row['형태소']):
            list_english.append(index)

    df_remove_english = df_komoran.drop(list_english)


    ## 숫자 제거
    for index, row in df_remove_english.iterrows():
        if re.match(pattern_number, row['형태소']):
            list_number.append(index)

    df_remove_number = df_remove_english.drop(list_number)


    ## 기호 제거
    for index, row in df_remove_number.iterrows():
        if re.match(pattern_symbol, row['형태소']):
            list_symbol.append(index)

    df_remove_symbol = df_remove_number.drop(list_symbol)


    ## 공백 제거
    for index, row in df_remove_symbol.iterrows():
        if re.match(pattern_empty, row['형태소']):
            list_empty.append(index)

    df_data = df_remove_symbol.drop(list_empty)

    # df_data['형태소'].unique()

    ## 미사용 형태 주석 제거
    using_pos = ['NP', 'NNG', 'XSA', 'EP', 'EF', 'JX', 'NNP', 'JKB', 'XSV', 'EC', 'VX', 'ETM', 'XSN', 'VCP', 'VV', 'VA', 'JKS', 'MAG', 'XR', 'JC', 'NA', 'VCN', 'MM', 'ETN', 'MAJ', 'JKG', 'JKC', 'IC', 'NNB', 'NR', 'JKO', 'XPN', 'JKQ']

    df_data = df_data[df_data['주석'].isin(using_pos)]

    # print(df_data['주석'].unique())
    
    ## stopwords 리스트 생성
    txt_tmp = open("./koreanStopwords.txt", 'r', encoding="utf-8")

    stopwords = []
    while True:
        line = txt_tmp.readline()
        if not line: break
        form, pos, _ = line.split()
        stopwords.append([form, pos])

    txt_tmp.close()

    ## stopwords 제거
    for f, p in stopwords:
        df_data = df_data[~((df_data["형태소"] == f) & (df_data["주석"] == p))]

    return df_data

In [99]:
## test 
result = tagging_pos_input_data("음 아안녕하세요. 전 저는 <name1>입니다. 저는 학생입니다. <name2>대학교 학교에 국 한국어를 배워요. 저는 지금? 좋아요. 어 제가쪽이 음 네 명이에요. 아버지하고 어머니하고 누나하고 저이에요. 아버지는 선생님이에요. 음 아버지는 지금? 좋아요. 어 어머니 어머니는 요리사이에요 예요(이에요). 엄 으음 뜨 뜨라 뜨라 드 드라마를 좋아요. 어 누나 누나 누나가 회사원이에요. 저 저 운도를? 해요. 아주 좋아해요.")

result

,형태소,주석
0,음,IC
1,아,IC
2,안녕하세요,IC
5,저,NP
6,는,JX
...,...,...
118,를,JKO
121,아요,EF
123,아주,MAG
124,좋아하,VV


# 단어 사전 기반 점수 

### 사전 데이터 전처리

In [5]:
#판다스로 어휘 문법 등급 목록 가져오기
import pandas as pd
df = pd.read_excel("./2017년 국제 통용 한국어 표준 교육과정 적용 연구(4단계) 어휘, 문법 등급 목록.xlsx", sheet_name = [0,1])

#어휘와 문법의 구성 요소만 확인할 수 있는 데이터프레임 생성
df_voca = pd.concat([df[0]['어휘'], df[1]['대표형']])

#확인
df_voca

0          가게
1        가격02
2        가구02
3         가깝다
4        가다01
        ...  
331      이라고는
332    -는다던가1
333     -으래서야
334     -으리라고
335      -자면2
Length: 10971, dtype: object

In [6]:
#값 내 숫자 제거
df_voca = df_voca.replace(r'\d+', '', regex=True)

#숫자 제거 확인
df_voca

0         가게
1         가격
2         가구
3        가깝다
4         가다
       ...  
331     이라고는
332    -는다던가
333    -으래서야
334    -으리라고
335      -자면
Length: 10971, dtype: object

In [7]:
#코모란 설치
!pip install PyKomoran

!python3 --version

Python 3.11.4


In [9]:
#코모란 라이브러리 가져오기 및 확인
from PyKomoran import *
komoran = Komoran("EXP")
print(komoran.get_plain_text("KOMORAN은 한국어 형태소 분석기입니다."))

KOMORAN/SL 은/JX 한국어/NNP 형태소/NNP 분석기/NNG 이/VCP ㅂ니다/EF ./SF


In [11]:
#한국어 불용어 사전 가져오기
stop = pd.read_table("./koreanStopwords.txt", names =["단어", "형태소", "수치"])
print(stop)

    단어  형태소        수치
0    이  VCP  0.018280
1    있   VA  0.011699
2    하   VV  0.009774
3    것  NNB  0.009733
4    들  XSN  0.006898
..  ..  ...       ...
95   원  NNB  0.000492
96   잘  MAG  0.000491
97  통하   VV  0.000487
98  소리  NNG  0.000486
99   놓   VX  0.000483

[100 rows x 3 columns]


In [12]:
#불용어 사전에서 필요한 정보만 추출하기
stop['단어']

0      이
1      있
2      하
3      것
4      들
      ..
95     원
96     잘
97    통하
98    소리
99     놓
Name: 단어, Length: 100, dtype: object

In [29]:
#어휘 문법 사전의 구성 요소들을 형태소 형태로 변경 + 불용어 제거
stopwords = [[stop['단어'], '-']]

voca_list = []

for w in df_voca:
  newdf = komoran.get_morphes_by_tags(w, tag_list=['NP', 'NNG', 'XSA', 'EP', 'EF', 'JX', 'NNP', 'JKB', 'XSV', 'EC', 'VX', 'ETM', 'XSN', 'VCP', 'VV', 'VA', 'JKS', 'MAG', 'XR', 'JC', 'NA', 'VCN', 'MM', 'ETN', 'MAJ', 'JKG', 'JKC', 'IC', 'NNB', 'NR', 'JKO', 'XPN', 'JKQ'])

  for w in newdf:
    if w not in stopwords:
      voca_list.append(w)

#형태소 형태로 전환된 새로운 어휘 문법 데이터프레임 생성
vocadf = pd.Series(voca_list)

#확인
print('상위 20개:\n',vocadf.head(20))
print('상위 20개:\n',vocadf.tail(20))

#데이터 프레임 저장
vocadf.to_csv('./vocadf.csv', index=False)

상위 20개:
 0       가
1       게
2      가격
3      가구
4      가깝
5       다
6       가
7       다
8     가르치
9       다
10     가방
11     가볍
12      다
13     가수
14     가요
15    가운데
16      가
17      을
18     가장
19     가족
dtype: object
상위 20개:
 15260    으리라는
15261       를
15262      막론
15263       하
15264       고
15265       이
15266       어
15267      치우
15268       다
15269       이
15270     는다는
15271     이라고
15272       는
15273       이
15274    는다던가
15275      으래
15276       서
15277      어야
15278    으리라고
15279      자면
dtype: object


### 사전 기반 발음 점수 Generator 모듈 (50점 만점의 점수 리턴)

In [73]:
# vocafd에서 word를 찾아서 있으면 1, 없으면 0 리턴함
def get_pronunciation_score_for_word_by_vocadf(word):
    return vocadf.str.contains(word).any()

# 사전 기반 발음 점수 -> 50점 만점
def get_pronunciation_score_by_vocadf(wordList):
    matchedCount = 0
    for word in wordList:
        matchedCount += get_pronunciation_score_for_word_by_vocadf(word)
    return (1-matchedCount/len(wordList))*50



print("당신의 사전 기반 발음 점수는: ")
print(score)

print("당신의 사전 기반 발음 점수는: ")
print(score2)


당신의 사전 기반 발음 점수는: 
33.333333333333336
당신의 사전 기반 발음 점수는: 
50.0


# 오타 기반 발음 점수 

In [1]:
! pip3 install symspellpy-ko


[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: /usr/local/opt/python@3.11/bin/python3.11 -m pip install --upgrade pip


In [2]:
from symspellpy_ko import KoSymSpell, split_syllables, join_jamos, Verbosity

def sym_spell():
    sp = KoSymSpell()
    sp.load_korean_dictionary(decompose_korean=True, load_bigrams=True)
    return sp

def test_lookup(sym_spell, term):
    lookup = sym_spell.lookup(term, Verbosity.CLOSEST)[0]
    assert lookup.term == "안녕하세요"
    assert lookup.distance == 1

In [3]:
sp = sym_spell()

lookup = sp.lookup("안뇽하십니까", Verbosity.CLOSEST)[0].term

print(lookup)

안녕하십니까


### 오타 기반 발음 점수 Generator 모듈 (50점 만점의 점수 리턴)

In [86]:
# word를 받아서 word의 오타를 제거하고, 제거된 char의 비율을 계산함. 
def get_pronunciation_score_by_correction_for_word(word):
    try :
        fixed_text = sp.lookup(word, Verbosity.CLOSEST)[0].term
        # count how many characters are different
        count = 0
        for i in range(len(word)):
            if word[i] not in fixed_text:
                count += 1
        score = (1 - count/len(word)) 
        # print(fixed_text)
    except :
        # print("Cannot find a proper text")
        return 0

    return score
    

# 리스트를 받아서 리스트 내 아이템 하나씩 스코어 함수 호출. (50점 만점)
def get_pronunciation_score_by_correction(list):
    score = 0
    for word in list:
        score += get_pronunciation_score_by_correction_for_word(word)
    return score/len(list)*50


get_pronunciation_score_by_correction(["아야", "맛난거"])

33.333333333333336

두 점수 종합하는 function

In [118]:
# Komoran 준비 ( Java 에러 종종 나서 분리)
komoran = Komoran("EXP")


In [112]:
def get_pronunciation_score(text):
    list_pos_tagged = tagging_pos_input_data(komoran, text)['형태소']
    tokenized_list = text.split(' ')
    # print(tokenized_list)
    score_by_voca = get_pronunciation_score_by_vocadf(list_pos_tagged)
    score_by_correction = get_pronunciation_score_by_correction(tokenized_list)


    print(f"단어 사전 기반 점수: {score_by_voca}")
    print(f"오타 교정 기반 점수: {score_by_correction}")
    print(f"종합 발음 점수: {score_by_voca+score_by_correction}")

    return score_by_voca+score_by_correction


get_pronunciation_score ("음 아안녕하세요. 전 저는 <name1>입니다. 저는 학생입니다. <name2>대학교 학교에 국 한국어를 배워요. 저는 지금? 좋아요. 어 제가쪽이 음 네 명이에요. 아버지하고 어머니하고 누나하고 저이에요. 아버지는 선생님이에요. 음 아버지는 지금? 좋아요. 어 어머니 어머니는 요리사이에요 예요(이에요). 엄 으음 뜨 뜨라 뜨라 드 드라마를 좋아요. 어 누나 누나 누나가 회사원이에요. 저 저 운도를? 해요. 아주 좋아해요.")
get_pronunciation_score ("어렸얼 어렸을 때 부짜 되고 싶어요")
get_pronunciation_score ("안녕하세요. 저는 학생입니다. 대학교에서 한국어를 배워요. 저는 기분이 좋아요")



Py4JJavaError: An error occurred while calling o131687.init.
: java.lang.OutOfMemoryError: GC overhead limit exceeded
